<a href="https://colab.research.google.com/github/komazawa-deep-learning/komazawa-deep-learning.github.io/blob/master/2022notebooks/2022_1028komazawa_neural_networks_primer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. ニューラルネットワークモデルの定義

以下に [PyTorch](https://pytorch.org/) を⽤いた，最も簡単なニューラルネットワークの定義を示します。

近年のニューラルネットワークモデルを構築するためのライブラリの特徴として， **自動微分** が挙げられます。
自動微分により，**誤差逆伝播法 back propagation** のための微分計算部分をコーディングする手間が不要になります。

このため，ニューラルネットワークモデルを作成する場合には，モデルのプロトタイプ (雛形) を定義する部分 `class` の内部にある関数定義 `def` は，最低二つあれば良いことになります。

二つとは，`__init__()` と `forward()` です。
`__init__()` 関数は初期化を担当し，`forward()` 関数は，前向きの処理，すなわち，入力データを処理して出力するまでを記述することになります。
学習に必要な `backward()` 関数は，上で書いた自動微分が自動的に計算されるので，不要となります。


```python
import torch
import torch.nn as nn

class perceptron(nn.Module):
    def __init__(self, in_features=2, out_features=1):
        super().__init__()
        self.layer = nn.Linear(in_featuers, out_features)
        self.act_f = nn.Sigmoid()
        
    def forward(self, data):
        out = self.act_f(self.layer(data))
         return out
    
network = perceptron()
   
print(network.state_dict())
print(network)
```

In [ ]:
import torch
import torch.nn as nn

class perceptron(nn.Module):
    def __init__(self, in_features=2, out_features=1):
        super().__init__()
        self.layer = nn.Linear(in_features=in_features, out_features=out_features)
        self.act_f = nn.Sigmoid()

    def forward(self, X):
        X = self.layer(X)
        X = self.act_f(self.layer(X))
        return X

net = perceptron()
print(f'net.eval():{net.eval()}')
print(f'net.state_dict():{net.state_dict()}')

# 2. 活性化関数

In [ ]:
# グラフ内の表記に日本語を使うための準備
try:
    import japanize_matplotlib
except ImportError:
    !pip install japanize_matplotlib
    import japanize_matplotlib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.special import expit as sigmoid

x = np.linspace(-2,2)
plt.plot(x, sigmoid(x), label="シグモイド関数")
plt.plot(x, np.tanh(x), label="ハイパータンジェント")
plt.plot(x, np.clip(x, 0, 6), label="ReLU6")
plt.legend()
plt.grid()
plt.show()

# 3. 損失関数

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

epsilon = 10 ** -4
print(epsilon)
x = np.linspace(0+epsilon, 1-epsilon)

plt.grid()   # マス目を表示するため
plt.plot(x, - np.log(x), label="$-\log(x)$", c="red")
plt.plot(x, - np.log(1-x), label="$-log(1-x)$", c="green")
plt.xlabel("確率 $p(x)$")
plt.ylabel("損失値 $\ell$")
plt.legend()
plt.show()

bce = x * np.log(x) + (1-x) * np.log(1-x)
plt.grid()   # マス目を表示するため
plt.plot(x, bce)
plt.title("交差エントロピー")
plt.xlabel("確率 $p(x)$")
plt.ylabel("損失値 $\ell$")
plt.show()

# 4. パーセプトロン

In [ ]:
import torch
import torch.nn as nn

class perceptron(nn.Module):
    def __init__(self, in_features=2, out_features=1):
        super().__init__()
        self.layer = nn.Linear(in_features=in_features, out_features=out_features)
        self.act_f = nn.Sigmoid()

    def forward(self, X):
        X = self.layer(X)
        out = self.act_f(X)
        return out

In [ ]:
net = perceptron() # モデルの実体化（インスタンス化）
loss_f = nn.MSELoss()  # 損失関数の 定義

import torch.optim as optim
optimizer = optim.SGD(net.parameters(), lr=0.01) # 最適化の定義

In [ ]:
X = torch.Tensor([[0,0],[0,1],[1,0],[1,1]])
y = torch.Tensor([[0],[1],[1],[1]])
print(X,y)

In [ ]:
net = perceptron()
loss_f = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001)
net.train()
iter_max = 10 ** 3
interval = iter_max >> 2
for i in range(iter_max):
    pred = net(X)
    loss = loss_f(pred, y)
    if (i % (iter_max >> 2)) == 0:
        print(f'{i:<5d} 損失: {loss.detach().numpy():.3f}')
    loss.backward() # Updating gradients
    optimizer.step()